In [1]:
import numpy as np
from scipy import *
import os
import h5py
import keras as k
from keras.layers import Dense, Activation
from keras.optimizers import SGD
from keras.models import Sequential
from keras.losses import categorical_crossentropy
from sklearn.utils import shuffle
from keras.utils import np_utils
import matplotlib.pyplot as plt
import scipy.optimize as opt
from skimage import data, color
from skimage.transform import rescale, resize, downscale_local_mean


%matplotlib inline

Using TensorFlow backend.


In [2]:
# Setting directory paths 
root_dir = os.getcwd()

# Loading data
data = h5py.File(r'C:\\Users\\fires\\Desktop\\CSIRO\\thursday\\Logistic Regression\\data.h5', 'r')

images = np.asarray(data['images'])
labels = data['labels']

In [3]:
# Define function to crop off 'cut' number of pixels on both sides on both axixs
def crop_center(img,cut):
    y = np.shape(img)[1]
    x = np.shape(img)[2]
    if x != y:
        print ("The image is not a perfect sqaure. This is bad. Fix it ")
        
    start = cut
    end = x-cut
    return img[:, start:end,start:end]

In [4]:
# Reducing Image Size
images_cut = crop_center(images,66)
images_cut = np.asarray(images_cut)

print (images_cut.shape)

(356, 135, 135)


In [5]:
# Defining rescaling function
def rescale_imgs(images, factor):
    count =  0
    for i in range(0 ,images.shape[0]):
        img_scaled = expand_dims(rescale(images[i, :, :], 1.0 / float(factor)), axis = 0)
        if count == 0:
            imgs = img_scaled
        else:
            imgs = np.vstack((imgs, img_scaled))
        count += 1
                             
    return imgs

In [6]:
images_small = (rescale_imgs(images_cut, 5))

C:\Users\fires\Anaconda3\envs\CSIRO\lib\site-packages\skimage\transform\_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
C:\Users\fires\Anaconda3\envs\CSIRO\lib\site-packages\skimage\transform\_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
C:\Users\fires\Anaconda3\envs\CSIRO\lib\site-packages\skimage\transform\_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
C:\Users\fires\Anaconda3\envs\CSIRO\lib\site-packages\skimage\transform\_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
C:\Users\fir

C:\Users\fires\Anaconda3\envs\CSIRO\lib\site-packages\skimage\transform\_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
C:\Users\fires\Anaconda3\envs\CSIRO\lib\site-packages\skimage\transform\_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
C:\Users\fires\Anaconda3\envs\CSIRO\lib\site-packages\skimage\transform\_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
C:\Users\fires\Anaconda3\envs\CSIRO\lib\site-packages\skimage\transform\_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
C:\Users\fir

C:\Users\fires\Anaconda3\envs\CSIRO\lib\site-packages\skimage\transform\_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
C:\Users\fires\Anaconda3\envs\CSIRO\lib\site-packages\skimage\transform\_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
C:\Users\fires\Anaconda3\envs\CSIRO\lib\site-packages\skimage\transform\_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
C:\Users\fires\Anaconda3\envs\CSIRO\lib\site-packages\skimage\transform\_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
C:\Users\fir

C:\Users\fires\Anaconda3\envs\CSIRO\lib\site-packages\skimage\transform\_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


In [7]:
print (np.shape(images_small))

(356, 27, 27)


In [8]:
# Reshape data
img = np.reshape(images_small, (-1, 729))
labels = np.reshape(labels, (-1,1))

In [9]:
##### This section is horrendous, I know #####


# Normalising the number of examples for each class
nb_FRI = int(np.asarray((np.where(labels == False))).size/2)
nb_FRII = int(np.asarray((np.where(labels == True))).size/2)

# Defining the maximum number of samples for each class
if nb_FRI >= nb_FRII:
    cutoff = nb_FRII
elif nb_FRI < nb_FRII:
        cutoff = nb_FRI

# seperating images and labels by class
FRI = img[:nb_FRI, :]
FRII = img[nb_FRI:, :]
FRI_labels = labels[:nb_FRI]
FRII_labels = labels[nb_FRI:]

# Slicing off excess samples
FRI = FRI[:cutoff, :]
FRI_labels = FRI_labels[:cutoff]

# Putting classes back together
normalized_img = np.vstack((FRI, FRII))
normalized_labels = np.vstack((FRI_labels, FRII_labels ))

# Randomizing the order of the rows
normalized_img, normalized_labels = shuffle(normalized_img, normalized_labels)

# converting to arrays
normalized_img = np.asarray(normalized_img)
normalized_labels = np.asarray(normalized_labels)

# Coverting Bools into binary values
labels = []

for i, value in enumerate(normalized_labels):
    if value == False:
        labels.append(0)
    elif value == True:
        labels.append(1)
    else:
        print ("error")

labels = np.asarray(labels)
labels = np.reshape(labels, (-1,1))

In [10]:
#imgs = np.reshape(normalized_img, (-1, 27, 27))

#for n in range(0, imgs.shape[0]):
   #plt.imshow(imgs[n,:,:], cmap='gray', shape=(27, 27))
   #print (n)
   #print (normalized_labels[n])
   #plt.show()

In [11]:
# Split data into training and testing sets

ratio = 0.8

def format_data(images, labels):
    split = round(ratio*images.shape[0])
    
    train_x = images[ :split, :]
    test_x = images[split:, :]
    train_y = labels[ :split]
    test_y = labels[split:]
    
    return train_x, test_x, train_y, test_y


train_x, test_x, train_y, test_y = format_data(normalized_img, normalized_labels)

In [12]:
# Defining Sigmoid Function
def sigmoid(z):
    g = 1.0/(1.0+np.exp(-z))
    return g

# Defining logistic loss function
def log_lost(theta, x, y):
    m = x.shape[0]
    hypothesis = sigmoid(np.dot(x, theta))
    loss = -np.sum(y.T.dot(np.log(hypothesis)) + (1-y.T).dot(np.log(1-hypothesis)))/m
    return loss

def  grad(theta, x, y):
    #this returns a vector, which is the gradient of the cost function (partial derivatives of the cost function)
    m = x.shape[0]
    hypothesis = sigmoid(np.dot(x, theta))
    
    grad = ((hypothesis-y).T.dot(x))/m
    grad = grad.mean(axis = 0)
    
    assert grad.shape == (730,), grad.shape
    return grad    

# Defining prediction function 
def predict(theta, x):
    p_1 = sigmoid(x.dot(theta))
    return np.reshape((p_1 >= 0.5).astype(int), (-1,1))

# Defining accuracy function 
def accuracy(predictions, test_y):
    if predictions.shape != test_y.shape:
        print ("Error")
    else:
        test_y[test_y == True] = 1
        test_y[test_y == False] = 0

        correct_pred = 0

        for i in range(test_y.shape[0]):
            if test_y[i] == predictions[i]:
                correct_pred += 1

        acc = correct_pred/test_y.shape[0]
    return acc

In [13]:
def format_data(data):
    # m= number of training data, n= number of features
    m = data.shape[0]
    n = data.shape[1]

    #Add bias
    data = np.hstack((np.ones((m, 1)), data)) 
    
    return data


#initiate values for the inital theta (taking into acount bias)
initial_theta = np.zeros((train_x.shape[1] + 1,))

In [14]:
train_x = format_data(train_x)
test_x = format_data(test_x)

print (train_x.shape)
print (initial_theta.shape)

(197, 730)
(730,)


In [15]:
Result = opt.fmin_bfgs(f=log_lost, x0 = initial_theta, 
                       args=(train_x, train_y), maxiter = 100, 
                       fprime = grad)

         Current function value: 0.670723
         Iterations: 14
         Function evaluations: 128
         Gradient evaluations: 116


In [16]:
g = predict(Result, test_x)
accuracy(g, test_y)

0.4489795918367347